In [3]:
import pandas as pd
import os
import glob
from tasks import compute_index
from tasks.post_process import compare_2020_2019_data_report

In [28]:
data_2019 = pd.read_csv('data/2019_archive/data.csv').assign(version='2019')
data_2020 = pd.read_csv('data/full_data/data.csv').assign(version='2020')

data = pd.concat([data_2019, data_2020], axis=0)

In [52]:
test = data.pivot(index=['Indicator', 'ISO', 'Year'], columns=['version'], values=['Value', 'Imputed', 'Corrected', 'From'])

In [55]:
test.loc['GN1', 'FRA']

Value           Imputed        Corrected           From        
version      2019      2020    2019   2020      2019   2020    2019    2020
Year                                                                       
1980     0.012758  0.012735   False  False      True   True  MANUAL  MANUAL
1981     0.013377   0.01334   False  False     False  False  MANUAL  MANUAL
1982     0.012955  0.012924   False  False     False  False  MANUAL  MANUAL
1983     0.012561  0.012545   False  False     False  False  MANUAL  MANUAL
1984     0.012086  0.012072   False  False     False  False  MANUAL  MANUAL
1985     0.012038  0.012032   False  False     False  False  MANUAL  MANUAL
1986     0.012011  0.011996   False  False     False  False  MANUAL  MANUAL
1987     0.011637  0.011627   False  False     False  False  MANUAL  MANUAL
1988     0.011635  0.011616   False  False     False  False  MANUAL  MANUAL
1989     0.011611  0.011587   False  False     False  False  MANUAL  MANUAL
1990     0.011633  0.011605   False  False     False  False  MANUAL  MANUAL
1991     0.011881  0.011863   False  False     False  False  MANUAL  MANUAL
1992     0.012084  0.012069   False  False     False  False  MANUAL  MANUAL
1993      0.01248  0.012475   False  False     False  False  MANUAL  MANUAL
1994     0.012985  0.012982   False  False     False  False  MANUAL  MANUAL
1995      0.01387  0.013877   False  False     False  False  MANUAL  MANUAL
1996      0.01446  0.014475   False  False     False  False  MANUAL  MANUAL
1997     0.015035  0.015049   False  False     False  False  MANUAL  MANUAL
1998      0.01567  0.015675   False  False     False  False  MANUAL  MANUAL
1999     0.015979  0.015975   False  False     False  False  MANUAL  MANUAL
2000     0.016472  0.016466   False  False     False  False  MANUAL  MANUAL
2001      0.01694  0.016924   False  False     False  False  MANUAL  MANUAL
2002     0.016781  0.016771   False  False     False  False  MANUAL  MANUAL
2003     0.016729  0.016728   False  False     False  False  MANUAL  MANUAL
2004     0.017258  0.017251   False  False     False  False  MANUAL  MANUAL
2005     0.017993  0.017971   False  False     False  False  MANUAL  MANUAL
2006     0.018902  0.018913   False  False     False  False  MANUAL  MANUAL
2007      0.01926  0.019283   False  False     False  False  MANUAL  MANUAL
2008     0.019735  0.019776   False  False     False  False  MANUAL  MANUAL
2009     0.020784  0.020812   False  False     False  False  MANUAL  MANUAL
2010     0.021625  0.021638   False  False     False  False  MANUAL  MANUAL
2011     0.022351  0.022371   False  False     False  False  MANUAL  MANUAL
2012      0.02228  0.022317   False  False     False  False  MANUAL  MANUAL
2013     0.022397  0.022403   False  False     False  False  MANUAL  MANUAL
2014      0.02244  0.022437   False  False     False  False  MANUAL  MANUAL
2015     0.022441  0.022429   False  False     False  False  MANUAL  MANUAL
2016     0.022024   0.02202   False  False     False  False  MANUAL  MANUAL
2017     0.021229  0.021233   False  False     False  False  MANUAL  MANUAL
2018     0.021229  0.020047    True  False     False  False  MANUAL  MANUAL
2019     0.021229  0.019435    True  False     False  False  MANUAL  MANUAL
2020     0.021229  0.019435    True   True     False  False  MANUAL  MANUAL

In [4]:
info_df = compare_2020_2019_data_report()

In [7]:
info_df

,%_imputed_19,%_imputed_20,%_outliers_19,%_outliers_20,From_19,From_20,earliest_year_19,earliest_year_20,latest_year_with_imputation_19,latest_year_with_imputation_20,latest_year_without_imputation_19,latest_year_without_imputation_20,n_points_19,n_points_20
Indicator,,,,,,,,,,,,,,
AB1,0.28,0.24,2.47,2.59,SDG API,SDG API,2000,2000,2020,2020,2020,2020,2835,3360
AB2,1.03,0.93,3.91,3.41,SDG API,SDG API,2000,2000,2020,2020,2020,2020,3969,3990
AB3,5.86,5.86,4.39,4.62,WB API,WB API,1998,1998,2020,2020,2020,2020,4761,4761
BE1,0.00,0.00,0.57,0.44,SDG API,SDG API,2000,2000,2020,2020,2020,2020,4599,4977
BE2,13.34,7.97,2.54,2.89,WB API,WB API,1990,1990,2020,2020,2016,2018,6448,6603
BE3,61.90,61.90,4.48,4.51,SDG API,SDG API,2000,2000,2020,2020,2020,2020,4305,4305
CV1,0.00,0.00,2.12,1.91,SDG API,SDG API,1993,1993,2020,2020,2020,2020,6328,6804
CV2,61.90,57.14,6.44,6.86,MANUAL,MANUAL,2000,2000,2020,2020,2019,2020,3822,3822
CV3,85.76,85.76,0.00,0.00,WB API,WB API,2000,2000,2020,2020,2018,2018,4431,4431


In [6]:
info_df.to_csv('data/results/2020_2019_report.csv')